In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
from functools import reduce

In [2]:
req = requests.get("https://www.open-medicaments.fr/api/v1/medicaments?query=paracetamol")

In [3]:
pref = "https://www.open-medicaments.fr/api/v1/medicaments/"

In [18]:
def get_qty(code):
    j = json.loads(requests.get(pref+code).text)
    #on trouve le nombre de boite et la contenance de celles ci
    liste = re.findall("\d+",j['presentations'][0]["libelle"])
    #on les multiplie
    return reduce(lambda x, y: x*y, [int(el) for el in liste])

In [5]:
import pandas as pd

In [6]:
df = pd.read_json(req.content)

In [7]:
data = df.denomination.str.extract(r'(\D*)(\d+) (\w+), (\w+)')

In [8]:
data["codeCIS"] = df.codeCIS

In [9]:
data.head()

,0,1,2,3,codeCIS
0,PARACETAMOL ZYDUS,500,mg,gélule,67445776
1,PARACETAMOL RATIOPHARM,1000,mg,comprimé,62721886
2,PARACETAMOL EG,1,g,comprimé,62772966
3,PARACETAMOL SANDOZ,500,mg,gélule,67034675
4,PARACETAMOL RATIOPHARM,500,mg,gélule,61223352


In [10]:
data[4] = data[1].where(data[2]=="mg", data[1]+"000").astype("int")

In [11]:
data["codeCIS"] = data.codeCIS.astype("str")

In [12]:
data.dtypes

0          object
1          object
2          object
3          object
codeCIS    object
4           int64
dtype: object

In [13]:
data["qte"] = data.codeCIS.apply(get_qty)

In [14]:
data.head()

,0,1,2,3,codeCIS,4,qte
0,PARACETAMOL ZYDUS,500,mg,gélule,67445776,500,16
1,PARACETAMOL RATIOPHARM,1000,mg,comprimé,62721886,1000,8
2,PARACETAMOL EG,1,g,comprimé,62772966,1000,8
3,PARACETAMOL SANDOZ,500,mg,gélule,67034675,500,16
4,PARACETAMOL RATIOPHARM,500,mg,gélule,61223352,500,16


In [15]:
data["equivalente_qte"] = data[4]*data.qte

In [16]:
data.columns = ["name", "dosage", "unite", "type","codeCIS","dosage_mg", "qte", "qte_equivalente"]

In [17]:
data

,name,dosage,unite,type,codeCIS,dosage_mg,qte,qte_equivalente
0,PARACETAMOL ZYDUS,500,mg,gélule,67445776,500,16,8000
1,PARACETAMOL RATIOPHARM,1000,mg,comprimé,62721886,1000,8,8000
2,PARACETAMOL EG,1,g,comprimé,62772966,1000,8,8000
3,PARACETAMOL SANDOZ,500,mg,gélule,67034675,500,16,8000
4,PARACETAMOL RATIOPHARM,500,mg,gélule,61223352,500,16,8000
5,PARACETAMOL ARROW,500,mg,comprimé,65085849,500,16,8000
6,PARACETAMOL AHCL,1,g,comprimé,61754805,1000,8,8000
7,PARACETAMOL ISOMED,500,mg,comprimé,60777377,500,16,8000
8,PARACETAMOL RATIOPHARM,500,mg,comprimé,62502166,500,16,8000
9,PARACETAMOL EVOLUGEN,1,g,comprimé,68830091,1000,8,8000
